In [1]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
def load_data():
    
    # load the dataset
    path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
    df = read_csv(path, header=None)
    
    return df

In [3]:
def create_data_labels(df):
    
    # split into input and output columns
    X, y = df.values[:, :-1], df.values[:, -1]
    # ensure all data are floating point values
    X = X.astype('float32')
    # encode strings to integer
    y = LabelEncoder().fit_transform(y)
    
    print('X.shape: {}'.format(X.shape))
    return X, y

In [4]:
def split_data(X, y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
    return X_train, X_test, y_train, y_test

In [11]:
def create_model(optimizer = 'adam'):

    # define model
    model = Sequential()
    model.add(Dense(10, activation = 'relu', kernel_initializer = 'he_normal', input_shape=(4,)))
    model.add(Dense(8, activation = 'relu', kernel_initializer = 'he_normal'))
    model.add(Dense(3, activation = 'softmax'))
    
    # compile the model
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [12]:
def define_param_grid():
    
    # define the grid search parameters
    batch_size = [10, 20, 40, 60]
    epochs = [25, 50, 100, 150]
    optimizers = ['sgd', 'adam']
    param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizers)
    
    return param_grid

In [13]:
def grid_search(model, param_grid, X_train, y_train):
    
    grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1, cv = 5)
    grid_result = grid.fit(X_train, y_train)
    
    # summarize results
    print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
    
    best_batch_size = grid_result.best_params_['batch_size']
    best_epochs = grid_result.best_params_['epochs']
    best_opt = grid_result.best_params_['optimizer']
    return best_batch_size, best_epochs, best_opt

In [14]:
def create_fit_best_model(batch_size, epochs, opt, X_train, y_train):
    
    model = create_model(optimizer = opt)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    return model

In [15]:
def main():
    
    df = load_data()
    X, y = create_data_labels(df)
    X_train, X_test, y_train, y_test = split_data(X, y)
    
    model = KerasClassifier(build_fn=create_model, verbose=0)
    param_grid = define_param_grid()
    
    # explore best parameters using GridSearch
    batch_size, epochs, opt = grid_search(model, param_grid, X_train, y_train)
    model = create_fit_best_model(batch_size, epochs, opt, X_train, y_train)
    # evaluate the model
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print('Accuracy on test data: %.3f' % acc)

In [16]:
if __name__ == '__main__':
    main()

X.shape: (150, 4)


<ipython-input-15-85a3d07243e2>:7: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Best: 0.945538 using {'batch_size': 20, 'epochs': 150, 'optimizer': 'adam'}
Accuracy on test data: 0.913
